# ROMS grid netcdf and S-coordinate
**Author: Jun Sasaki, Coded on September 5, 2020, Updated on September 6, 2020**<br>
- A sample code for manipulating `Projects/Sandy/Sandy_roms_grid.nc`
- S-coordinate is not included in grid netcdf, which should be specified.
- S-coordinate has two options specified by Vtransform. Surface elevation zeta is supposed to be 0.
- Reference for setting S-coordinate see [xaray-example](http://xarray.pydata.org/en/stable/examples/ROMS_ocean_model.html) and [Wiki ROMS](https://www.myroms.org/wiki/Vertical_S-coordinate).

In [ ]:
import numpy as np
from netCDF4 import num2date, date2num
import datetime
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib import colors as c
import xarray as xr
import hvplot.xarray
import holoviews as hv
from holoviews import opts
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Set plotting parameters

In [ ]:
z = 'mask_rho'  ### 'h'/'mask_rho'/'f'/'pm'/'pn'/'dndx'/'dmde'
cmap='magma_r'
cmap2 = ['darkgoldenrod', 'aqua']
frame_height=200; frame_width=200
line_color='lightgrey'; line_width=0.1
coastline='10m'  ### '10m'/'50m'/'110m'
proj = ccrs.LambertConformal(central_longitude=-73, central_latitude=36)  ### Do Not set geo=True when proj is used.
### tiles= 'CartoDark'/'CartoEco'/'CartoLight'/'CartoMidnight'/'EsriImagery'/'EsriNatGeo'/'EsriReference'/'ESRI'/
###        'EsriTerrain'/'EsriUSATopo'/'OSM'/'StamenLabels'/'StamenTerrain'/'StamenTerrainRetina'/'StamenToner'/
###        'StamenTonerBackground'/'StamenWatercolor'
tiles = 'StamenTerrain'
### geo=True: Geographic and assumed PlateCarree (Do Not set when proj is used.)
### Options for S-coordinate
Vtransform = 1  ## 1/2
smax = 10  ### num of S-coord
dsigma = 1.0/smax
s_rho_val = np.arange(-1.0+dsigma/2.0, 0.0, dsigma)
hc = 20  ### critical depth
s_rho_val

# Loading grid netcdf into xarray.Dataset and create S-coord and z-coord

In [ ]:
def sigma_to_z_rho(ds, s_rho_val):
    '''
    Creating z_rho (z coordinate value array at z_rho) using inver sigma-coordinate transform equation
    
    Args:
        grd (xarray.Dataset) : Grid
        itime (int) : Index value of time coordinate
    Returns:
        z_rho (ndarray) : z coordinate value array at (itime, s_rho[:], erho[:], xrho[:])
    '''

    z_rho = 0.0 + s_rho_val[0] * (ds['h'].values)
    #print(z_rho.shape)
    z_rho = z_rho[np.newaxis,:,:]  ### extend dimension
    #print(z_rho.shape)
    for k in np.arange(1, len(ds['s_rho'])):
        z_rho1 = 0.0 + ds['s_rho'][k].values * (ds['h'].values + 0.0)
        z_rho1 = z_rho1[np.newaxis,:,:]
        #print(z_rho1.shape)
        z_rho = np.vstack((z_rho, z_rho1))
    return z_rho

In [ ]:
dir_path = "../../Projects/Sandy/"
grid_nc = dir_path + 'Sandy_roms_grid.nc'
with xr.open_dataset(grid_nc) as ds:
    pass

def set_coords(ds, items):
    for item in items:
        ds.coords[item] = ds[item]
    return ds
items = ["eta_psi", "eta_rho", "eta_u", "eta_v", "xi_psi", "xi_rho", "xi_u", "xi_v"]
ds = set_coords(ds, items)
ds = ds.set_coords(["x_rho", "y_rho", "lat_rho", "lon_rho"])
ds['s_rho'] = xr.DataArray(s_rho_val, dims=["s_rho"])
#ds['s_rho'].attrs["long_name"] = "S-coord"
ds['z_rho'] = (("s_rho", "eta_rho", "xi_rho"), sigma_to_z_rho(ds, s_rho_val))
#ds['z_rho'] = xr.DataArray(sigma_to_z_rho(ds, s_rho_val), dims=["s_rho", "eta_rho", "xi_rho"])
ds['z_rho'].attrs['long_name'] = 'z-coord'
ds['z_rho'].attrs['units'] = 'm'
ds.coords['z_rho'] = ds['z_rho']
ds['depth'] = ds['z_rho']
#ds.set_coords(['z_rho'])
ds['s_rho'].attrs['long_name'] = 'S-coord'
ds

In [ ]:
### Grid number coords with specifying xlim and ylim
xlim=(20,60)
ylim=(20,50)
ds.hvplot.quadmesh(x='xi_rho', y='eta_rho', z=z , \
                   frame_height=frame_height, frame_width=frame_width, cmap=cmap2, \
                   xlim=xlim, ylim=ylim, colorbar=False, \
                   line_color=line_color, line_alpha=1, line_width=0.1)

In [ ]:
### Cartesian (x,y) coords
ds.hvplot.quadmesh(x='x_rho', y='y_rho', z=z , \
                   frame_height=frame_height, frame_width=frame_width, cmap=cmap2, colorbar=False, \
                   line_color=line_color, line_alpha=1, line_width=0.1)

In [ ]:
### Geographic coords without projection
ds.hvplot.quadmesh(x='lon_rho', y='lat_rho', z=z , \
                   frame_height=frame_height, frame_width=frame_width, cmap=cmap2, colorbar=False, \
                   line_color=line_color, line_alpha=1, line_width=0.2)

In [ ]:
### Geographic coords with tiles, however, coords values in hover are wrong
ds.hvplot.quadmesh(x='lon_rho', y='lat_rho', z=z , geo=True, tiles=tiles, \
                   frame_height=frame_height, frame_width=frame_width, cmap=cmap2, colorbar=False, \
                   coastline=coastline, \
                   line_color=line_color, line_alpha=1, line_width=0.2, alpha=0.5)

In [ ]:
### Geographic coords with specified projection, however, coords values in hover are wrong
ds.hvplot.quadmesh(x='lon_rho', y='lat_rho', z=z , projection=proj, \
                   frame_height=frame_height, frame_width=frame_width, cmap=cmap2, colorbar=False, \
                   coastline=coastline, \
                   line_color=line_color, line_alpha=1, line_width=0.2)

In [ ]:
ds.hvplot.quadmesh(groupby=['eta_rho'], x='xi_rho', y='s_rho', z='depth', \
                   frame_height=frame_height, frame_width=frame_width, cmap='magma', \
                   ylim=(-1, 0), line_color='aqua', line_alpha=1, line_width=0.1)

In [ ]:
ds.hvplot.quadmesh(groupby=['eta_rho'], x='xi_rho', y='z_rho', z='depth', \
                   frame_height=frame_height, frame_width=frame_width, cmap='magma', \
                   ylim=(-5500, 0), line_color='aqua', line_alpha=1, line_width=0.1)

In [ ]:
ds.hvplot.quadmesh(groupby=['xi_rho'], x='eta_rho', y='s_rho', z='depth', \
                   frame_height=frame_height, frame_width=frame_width, cmap='magma', \
                   ylim=(-1, 0), line_color='aqua', line_alpha=1, line_width=0.1)

In [ ]:
ds.hvplot.quadmesh(groupby=['xi_rho'], x='eta_rho', y='z_rho', z='depth', \
                   frame_height=frame_height, frame_width=frame_width, cmap='magma', \
                   ylim=(-5500, 0), line_color='aqua', line_alpha=1, line_width=0.1)

In [ ]:
#hv.help(hv.QuadMesh)

# Plotting with matplotlib

In [ ]:
cMap = c.ListedColormap(['darkgoldenrod', 'aqua'])
ds[z].plot.pcolormesh(x='xi_rho', y='eta_rho', cmap=cMap, add_colorbar=False)

In [ ]:
cMap = c.ListedColormap(['darkgoldenrod', 'aqua'])
ds[z].plot.pcolormesh(x='lon_rho', y='lat_rho', cmap=cMap, add_colorbar=False)

In [ ]:
p=ds[z].plot(x='xi_rho', y='eta_rho', cmap=cMap, add_colorbar=False, \
             alpha=1, edgecolor='grey', linewidth=0.01)
#p.axes.set_xlim(40, 70)
#p.axes.set_ylim(30, 50)
p.figure.savefig('grid.png', dpi=300, bbox_inches='tight')

## Vertical sectional view in z coordinate

In [ ]:
ds['depth'].isel(eta_rho=0).plot.pcolormesh(x='xi_rho', y='s_rho', extend='both', cmap='magma')

In [ ]:
p=ds['depth'].isel(eta_rho=0).plot.pcolormesh(x='xi_rho', y='z_rho', extend='both', cmap='magma',\
                                              alpha=1, edgecolor='aqua', linewidth=0.01)
p.axes.set_facecolor('lightgray')
p.figure.savefig('grid_ez.png', dpi=300, bbox_inches='tight')

In [ ]:
ds['depth'].isel(xi_rho=30).plot.pcolormesh(x='eta_rho', y='s_rho', extend='both', cmap='magma')

In [ ]:
p=ds['depth'].isel(xi_rho=30).plot.pcolormesh(x='eta_rho', y='z_rho', extend='both', cmap='magma')
p.axes.set_facecolor('lightgray')
p.figure.savefig('grid_xz.png', dpi=300, bbox_inches='tight')